In [ ]:
%%capture

import warnings

warnings.filterwarnings("ignore")
import calitp_data_analysis.magics

import geopandas as gpd
import pandas as pd

from great_tables import GT
import branca.colormap as cm

import _report_utils
import _leg_district_grain_data_prep as _legislative_district_prep
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS
from shared_utils import webmap_utils

SHARED_GCS = GTFS_DATA_DICT.gcs_paths.SHARED_GCS

In [ ]:
import google.auth

credentials, project = google.auth.default()

import gcsfs

fs = gcsfs.GCSFileSystem()

In [ ]:
# Comment out and leave this cell right below pandas
# district = "AD 03"

In [ ]:
district_full_name = _legislative_district_prep.readable_district_name(district)

In [ ]:
%%capture_parameters
district, district_full_name

In [ ]:
operator_df = _legislative_district_prep.load_district_stats(district)

In [ ]:
operator_route_gdf = _legislative_district_prep.load_gtfs_data(operator_df)

In [ ]:
district_file_name = district.replace(" ","_").lower()

In [ ]:
# Files for webmaps
route_file = f"leg_district_{district_file_name}_routes"

# {district}

These are district summaries for [GTFS Digest](https://gtfs-digest--cal-itp-data-analyses.netlify.app/). 

In [ ]:
district_summary = _report_utils.district_stats(
    operator_df, 
    "legislative_district"
)

In [ ]:
summary_table1 = (GT(
    district_summary.drop(
        columns = ["arrivals_per_stop", "trips_per_operator"]
    ).pipe(
        _report_utils.transpose_summary_stats, 
        district_col = "legislative_district"
    )
)
 .fmt_integer(columns="value")
 .cols_label(index="")
 .tab_header(title = f"{district_full_name} GTFS summary stats")
)

summary_table2 = (GT(
    district_summary[
        ["legislative_district", "arrivals_per_stop", "trips_per_operator"]
    ].pipe(
        _report_utils.transpose_summary_stats,
        district_col = "legislative_district"
    )
)
 .fmt_number("value", decimals=1)
 .cols_label(index="")
)

In [ ]:
display(summary_table1)
display(summary_table2)

## GTFS Stats by Operator

In [ ]:
color_map = cm.linear.Spectral_11.scale()

In [ ]:
try:
    district_map = webmap_utils.set_state_export(
    operator_route_gdf,
    subfolder = "caltrans_legislative_district_digest/",
    filename=route_file,
    map_title="Legislative District Route Map",
    overwrite = True
    )
except:
    pass

In [ ]:
try:
    webmap_utils.render_spa_link(district_map["spa_link"], text=f"Open Routes for all Operators Map in {district_full_name}")
except:
    pass

In [ ]:
try:
    webmap_utils.display_spa_map(district_map["spa_link"])
except:
    pass

In [ ]:
gtfs_table_df = _legislative_district_prep.create_gtfs_table(operator_df)

In [ ]:
string_cols = gtfs_table_df.select_dtypes(include="object").columns.tolist()

In [ ]:
int_columns = [
    c
    for c in gtfs_table_df.columns
    if c not in ["Operator Service Miles", "Avg Arrivals per Stop"]
    and (c not in string_cols)
]

In [ ]:
gtfs_table = (
    GT(gtfs_table_df.sort_values("# Trips", ascending=False))
    .fmt_integer(
        columns = [
            c for c in gtfs_table_df.columns if c not in 
         ["Operator Service Miles", "Avg Arrivals per Stop"]
            and (c not in string_cols)
        ])
    .fmt_number(
        columns = ["Operator Service Miles", "Avg Arrivals per Stop"], 
        decimals = 1)
    .data_color(
        columns=["# Trips", "Avg Arrivals per Stop"],
        palette=["white", "green"],
        na_color="lightgray")
    .tab_header(
         title = f"{district_full_name}",
         subtitle = "Daily GTFS schedule statistics by operator")
    .cols_align(columns = [c for c in gtfs_table_df.columns if c not in ["Organization", "Transit Operator"]],
        align="center")
    )


In [ ]:
gtfs_table = _report_utils.great_table_formatting(gtfs_table)
gtfs_table